In [1]:
# importing necessary modules.
import requests
import json
import pandas as pd
import numpy as np
import config2 as cfg
from sqlalchemy import create_engine

In [2]:
# creating a connection to an aws rds postgres cloud database.
rds_connection_string = f'postgres:{cfg.password}@test-db.cy2enoewwvsi.us-east-2.rds.amazonaws.com:5432/stocks_db'
engine = create_engine(f'postgres://{rds_connection_string}')

In [3]:
# checking out the SandP_500 table.
ticker_symbols = []
sandp = pd.read_sql_query('SELECT * FROM sandp2', con=engine)
for symbol in sandp['symbol']:
    ticker_symbols.append(symbol)

In [4]:
# creating empty list.
industry_list = []

# creating empty list.
sector_list = []

# looping through ticker_symbols list.
for ticker in ticker_symbols:
    
    # try except statement to ignore missing data.    
    try:

        # obtaining the json data for sector and industry type from financialmodelingprep.          
        response = requests.get(f'https://financialmodelingprep.com/api/v3/company/profile/{ticker}').json()

        # getting industry data.
        industry_list.append(response['profile']['industry'])
        
        # getting sector data.
        sector_list.append(response['profile']['sector'])
    
    # try except statement to ignore missing data.
    except:
        pass

In [5]:
# creating an empty list.
market_cap_list = []

# looping through ticker_symbols list.
for ticker in ticker_symbols:
        
    # try except statement to ignore missing data.    
    try:
        
        # obtaining the json data for market cap from financialmodelingprep.   
        response2 = requests.get(f'https://financialmodelingprep.com/api/v3/enterprise-value/{ticker}').json()
        
        # calculating average market cap value from 2009/2010.             
        market_cap = (int(response2['enterpriseValues'][-1]['Market Capitalization']) + int(response2['enterpriseValues'][-2]['Market Capitalization']))/2

        # appending market cap value to market cap list.          
        market_cap_list.append(market_cap)
    
    # try except statement to ignore missing data.    
    except:
        pass

In [6]:
# adding market cap to dataframe
sandp['market_cap'] = market_cap_list

In [7]:
# adding industry to dataframe.
sandp['industry'] = industry_list

In [8]:
# adding sector to dataframe.
sandp['sector'] = sector_list

In [9]:
# viewing resuts.
sandp.head()

,symbol,name,price_to_bookvalue,price_to_book,price_to_sales,price_to_earnings,receivables_turnover,price_to_free_cash_flow,price_to_operating_cash_flow,enterprise_value_multiple,...,asset_growth,book_value_per_share_growth,debt_growth,randd_expense_growth,sganda_expense_growth,percent_return_on_investment,above_below_sandp_return,market_cap,industry,sector
0,A,"Agilent Technologies, Inc.",2.413067,4.774367,1.451467,7.232900,7.218200,14.9343,11.489000,12.491404,...,0.098067,0.212733,0.076567,-0.024767,0.023367,215.407785,1,7.462700e+09,Medical Diagnostics & Research,Healthcare
1,AAL,"American Airlines Group, Inc.",-99.000000,-99.000000,0.100967,-99.000000,27.970767,-99.0000,2.242833,41.208570,...,-0.017600,-0.305567,0.064167,-99.000000,0.039567,492.976589,1,6.133809e+08,Airlines,Industrials
2,AAP,"Advance Auto Parts, Inc.",4.709967,5.153300,0.815500,14.602133,52.804800,9.5982,6.588500,0.514723,...,0.072700,0.004433,0.094767,-99.000000,0.066900,158.290852,1,1.912092e+09,Retail - Apparel & Specialty,Consumer Cyclical
3,AAPL,"Apple, Inc.",5.246733,4.813033,3.805867,17.821233,10.061767,15.6817,13.559600,0.419726,...,0.481267,0.487467,-99.000000,0.300633,0.270100,641.441749,1,9.151793e+10,Computer Hardware,Technology
4,ABBV,"AbbVie, Inc.",16.468700,-99.000000,3.019000,10.483600,2.426100,9.2123,8.745400,0.006801,...,-0.038200,-99.000000,-99.000000,0.255300,0.341850,127.493713,0,6.619065e+10,Drug Manufacturers,Healthcare


In [10]:
# appeending pandas dataframe to cloud postgres database.
sandp.to_sql(name='sandp2', con=engine, if_exists="replace", index=False)